In [ ]:
import json

with open('embeddings_output.json') as f:
    embed= json.load(f)



In [ ]:
import docx

In [14]:
len(embed[2]['embedding'])

384

# Generación de embeddings para sistema RAG
- **Objetivo**: Entregar embeddings generados a AI Devs para que lo implemente en sistema RAG del agente eCommerce
- **Proceso**: realizar desde el proceso de Chunking, evaluar los documentos generados para elegir el modelo, generar el embedding, evaluar resultados con implementación de Rag con OpenAI.

## Chunking
**Requisitos**
- *Tamaño óptimo:* 200-500 tokens por chunk
- *Overlap:* 50-100 tokens entre chunks consecutivos
- *Estrategias:*
  - Por párrafos (para políticas)
  - Por producto individual (para catálogos) (ya lo manejan ellos)
  - Por pregunta-respuesta (para FAQs)


In [19]:
import docx

ModuleNotFoundError: No module named 'docx'